# XFeat minimal inference example

涉及的函数:
1. 单张图像特征点检测和描述子计算:  `output = xfeat.detectAndCompute(tensor类型的图像, top_k)[0]`，输入包含了从该图像中获取的特征点、描述子和评分
2. 两张图像稀疏匹配： `图像1匹配点像素坐标<列表>, 图像2匹配点像素坐标<列表> = xfeat.match_xfeat(图像1, 图像2, top_k)`
3. 两张图像半稠密匹配：`图像1匹配点像素坐标<列表>, 图像2匹配点像素坐标<列表> = xfeat.match_xfeat_star(图像1, 图像2, top_k)`

还可用批处理模式，此时上面的输出可能略有改变

## Clone repository  改变工作目录

In [7]:
# 在shell中执行
# !cd /content && git clone 'https://github.com/verlab/accelerated_features.git'
# 改变当前的工作目录
# %cd /content/accelerated_features
%cd ~/accelerated_features/

## Test on simple input (sparse setting)

简单输入的测试，一次输入一个图像

In [8]:
import numpy as np
import os
import torch
import tqdm

from modules.xfeat import XFeat

xfeat = XFeat()

# 随机输入，生成一个(1,3,480,640)的张量, 代表一个大小为480x640的3通道图像
x = torch.randn(1,3,480,640)

# batch = 1, 对单张tensor类型的图像进行特征点检测和描述子计算，返回一个列表，列表的第一个元素是字典，包含了检测到的特征点、描述子和分数
output = xfeat.detectAndCompute(x, top_k = 4096)[0]
print("----------------")
print("keypoints: ", output['keypoints'].shape)
print("descriptors: ", output['descriptors'].shape)
print("scores: ", output['scores'].shape)
print("----------------\n")

## Stress test to check FPS on VGA (sparse setting)
对VGA进行压力测试，检查在稀疏设置下的每秒帧数（FPS）

蛟龙16, CPU AMD R7 6800h, GPU 3060  
480x640 230FPS/s  
1080x1920 55FPS/s  

In [16]:
x = torch.randn(1,3,480,640)

# Stress test
for i in tqdm.tqdm(range(100), desc="Stress test on VGA resolution"):
	output = xfeat.detectAndCompute(x, top_k = 4096)


## Test with batched mode (sparse)

批量模式的测试，一次输入4个图像

In [17]:
# Batched mode
x = torch.randn(4,3,480,640)
outputs = xfeat.detectAndCompute(x, top_k = 4096)
print("# detected features on each batch item:", [len(o['keypoints']) for o in outputs])

## Matches two images with built-in MNN matcher (sparse mode)

使用内置的MNN(最小化最近邻)匹配器匹配两幅图像，稀疏模式

In [18]:
# Match two images with sparse features
x1 = torch.randn(1,3,480,640)
x2 = torch.randn(1,3,480,640)
mkpts_0, mkpts_1 = xfeat.match_xfeat(x1, x2)

## Matches two images with semi-dense matching, and batched mode (batch size = 4) for demonstration purpose

使用半稠密和批处理(batch size = 4)匹配两幅图像

In [19]:
# Create 4 image pairs
x1 = torch.randn(4,3,480,640)
x2 = torch.randn(4,3,480,640)

#Obtain matches for each batch item
matches_list = xfeat.match_xfeat_star(x1, x2, top_k = 5000)
print('number of img pairs', len(matches_list))
print(matches_list[0].shape) # -> output is (x1,y1,x2,y2)